<a href="https://colab.research.google.com/github/McPatrik/Data-analyst---Projects./blob/main/rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задача.**
Произвести RFM анализ, то есть осуществить сегментацию базы клиентов на основании давности, частоты и суммы покупок. Это необходимо для выстраивания дальнейшей работы с клиентом на основе его сегмента, например предложение системы лояльности, новинок, email-рассылок и прочего.

Первоначально импортирует необходимые для работы библиотеки, pandas для обработки и анализа, numpy для математических вычислений, scikit-learn для использования алгоритма k-средних и стандартизации наших данных, также нам понадобиться plotly для отрисвки наших графиков.




# Данные для исследования
Файл для исследования был взят с сайта kaggle, ссылка на файл https://www.kaggle.com/datasets/carrie1/ecommerce-data?resource=download



Это транснациональный набор данных, который содержит все транзакции, имевшие место в период с 12.01.2010 по 12.09.2011 для британской и зарегистрированной немагазинной онлайн-розничной торговли. Компания в основном продает уникальные подарки на все случаи жизни. Многие клиенты компании являются оптовиками.
Прочтём и ознакомимся с данными,файл содержит такую информацию как номер счёта, код товара, описание товара, количество приобретенного товара, дата заказа, стоимость за единицу товара, id пользователя и страна. Данные не содержат пропусков. На каждого пользователя имеется 1 и более номера счётов, где один счёт - заказ.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import statistics
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/data.csv', encoding = 'ISO-8859-1')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Обработка данных

*   Используя группировку по id, посчитаем количество заказов, тем самым найдём частоту(frequency)
*   Найдём сумму товаров каждого совершенного заказа, а далее общую сумму покупок пользователя(monetary)
*   Посчитаем количество дней с последней покупки пользователя, где текущая - последняя дата покупки из датасета, тем самым найдем наш показатель давности(recency)



  



Рассмотрим такие показатели как уникальное количество счётов по отношению ко всем существующим счетам, дату первой покупки, а также последней(последняя дата покупки будет взята за текущую дату), количество стран,и количество уникальных CustomerID. Обратим внимание, что один клиент имеет несколько счётов.

In [6]:
df.agg({'InvoiceNo':['nunique', 'count'],
        'InvoiceDate':['min', 'max'],
        'CustomerID':'nunique',
        'Country':'nunique'})

,InvoiceNo,InvoiceDate,CustomerID,Country
nunique,25900.0,NaN,4372.0,38.0
count,541909.0,NaN,NaN,NaN
min,NaN,1/10/2011 10:04,NaN,NaN
max,NaN,9/9/2011 9:52,NaN,NaN


In [7]:
#frequency  кол-во заказов совершенным каждым пользователем
Frequency = df.groupby("CustomerID")['InvoiceNo'].nunique().reset_index()
# сумма заказанных товаров
df['sum_Of_goods'] = df['UnitPrice'] * df['Quantity']
#monetary
Monetary = df.groupby('CustomerID')['sum_Of_goods'].sum().reset_index()
#recency
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
#df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
last_date_order = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
current_date = df['InvoiceDate'].max()
last_date_order['Recency'] = current_date - last_date_order['InvoiceDate']
last_date_order['Recency'] = last_date_order['Recency'].dt.days

Создание результативного датафрейма, включающего в себя информацию полученную выше.

In [8]:
df_result = last_date_order.copy()
df_result = df_result.merge(Frequency, on='CustomerID')
df_result = last_date_order.copy() # !!! чёт тут какие-то задвоения по коду
df_result.drop(columns='InvoiceDate',inplace=True)
df_result = df_result.merge(Frequency, on='CustomerID')
df_result = df_result.merge(Monetary, on='CustomerID')
df_result.rename(columns={'InvoiceNo': 'Frequency', 'sum_Of_goods': 'Monetary'},inplace=True)

# Простая сегментация пользователей

Для начала разобьём пользователей на 3 группы по каждому признаку.
Для этого мы определим 2 границы значений по каждому признаку. А именно 33 и соответственно 66 процентиль, тем самым найдём 3 равные количественно группы.

In [9]:
first_quartile = df_result[['Recency', 'Frequency', 'Monetary']].quantile(0.33)
last_quartily = df_result[['Recency', 'Frequency', 'Monetary']].quantile(0.66)

Выведем полученные значения.

In [10]:
first_quartile,last_quartily

(Recency       23.0000
 Frequency      2.0000
 Monetary     369.2602
 Name: 0.33, dtype: float64,
 Recency        86.0000
 Frequency       4.0000
 Monetary     1119.7652
 Name: 0.66, dtype: float64)

In [11]:
df['InvoiceDate'].max(), df['InvoiceDate'].min()

(Timestamp('2011-12-09 12:50:00'), Timestamp('2010-12-01 08:26:00'))

Разница между первой и последней покупкой составляет:

In [12]:
df['InvoiceDate'].max() - df['InvoiceDate'].min()

Timedelta('373 days 04:24:00')

Выведем описание по нашим признакам, включая среднее, стандартное отклонение от среднего минимальное и максимальное значение и т.д

In [13]:
df_result[['Recency', 'Frequency', 'Monetary']].describe()

,Recency,Frequency,Monetary
count,4372.000000,4372.000000,4372.000000
mean,91.047118,5.075480,1898.459701
std,100.765435,9.338754,8219.345141
min,0.000000,1.000000,-4287.630000
25%,16.000000,1.000000,293.362500
50%,49.000000,3.000000,648.075000
75%,142.000000,5.000000,1611.725000
max,373.000000,248.000000,279489.020000


Пользователь по каждому признаку получит значение от 1 до 3, взависимости от числового показателя. Далее будет произведена конкатенация значений в единый(дополнительный) id группы присвоенный каждому из пользователей. В результате было получено 27 групп.

In [14]:
group_recency = (df_result['Recency'] > first_quartile['Recency']).astype(int) + (df_result['Recency'] > last_quartily['Recency']).astype(int) + 1
group_frequancy = (df_result['Frequency'] > first_quartile['Frequency']).astype(int) + (df_result['Frequency'] > last_quartily['Frequency']).astype(int) + 1
group_monetary = (df_result['Monetary'] > first_quartile['Monetary']).astype(int) + (df_result['Monetary'] > last_quartily['Monetary']).astype(int) + 1

In [15]:
df_result['group_recency'] = group_recency
df_result['group_frequancy'] = group_frequancy
df_result['group_monetary'] = group_monetary
df_result['id_group'] = df_result['group_recency'].astype(str) + df_result['group_frequancy'].astype(str) + df_result['group_monetary'].astype(str)

In [16]:
color_ = pd.DataFrame({'id_group': list(df_result['id_group'].unique())})
color_n = []
for i in range(1,len(color_)*10, 10):
  color_n.append(i)
color_['color'] = color_n

In [17]:
df_result = df_result.merge(color_, on='id_group')
df_result['color']

0         1
1         1
2         1
3         1
4         1
       ... 
4367    251
4368    251
4369    251
4370    261
4371    261
Name: color, Length: 4372, dtype: int64

In [18]:
df_result

,CustomerID,Recency,Frequency,Monetary,group_recency,group_frequancy,group_monetary,id_group,color
0,12346.0,325,2,0.00,3,1,1,311,1
1,12350.0,309,1,334.40,3,1,1,311,1
2,12353.0,203,1,89.00,3,1,1,311,1
3,12361.0,286,1,189.90,3,1,1,311,1
4,12373.0,310,1,364.60,3,1,1,311,1
...,...,...,...,...,...,...,...,...,...
4367,17440.0,147,5,177.10,3,3,1,331,251
4368,17912.0,309,6,295.61,3,3,1,331,251
4369,18037.0,153,6,70.02,3,3,1,331,251
4370,16321.0,71,7,230.70,2,3,1,231,261


In [37]:
a

,id_group,color,CustomerID
0,111,91,200
1,112,151,126
2,113,31,18
3,121,201,25
4,122,111,176
5,123,61,93
6,131,241,4
7,132,181,104
8,133,11,725
9,211,191,356


На графике отображено количество пользователей в каждой из групп, а также каким образом распределены группы по значения признака, как мы видим распределение групп по признакам выглядит весьма корректно.

In [36]:
a = df_result.groupby(['id_group', 'color'], as_index=False)['CustomerID'].nunique()
n = 6
seg = a.sort_values(by='CustomerID', ascending=False).id_group.values[:n]
need_seg = df_result[df_result['id_group'].isin(seg)]
color = a['id_group'].astype(int)

trace1 = go.Bar(x=a['id_group'], y=a['CustomerID'], name='Counts of buyers in groups', marker=dict(color= a['color']))

trace2 = go.Scatter(x = need_seg['Monetary'],
                    y = need_seg['Frequency'],
                    mode='markers',
                    marker=dict(color=df_result['color']), name = 'Monetary & Frequency',  hovertext=need_seg['id_group'])

trace3 = go.Scatter(x = need_seg['Monetary'],
                    y = need_seg['Recency'],
                    mode='markers',
                    marker=dict(color=df_result['color']), name = 'Monetary & Recency',  hovertext=need_seg['id_group'])

trace4 = go.Scatter(x = need_seg['Frequency'],
                    y = need_seg['Recency'],
                    mode='markers',
                    marker=dict(color=df_result['color']), name = 'Frequency & Recency', hovertext=need_seg['id_group'])

fig = make_subplots(rows=2, cols=2, subplot_titles=('Counts of buyers in groups', 'Monetary & Frequency', 'Monetary & Recency','Frequency & Recency'))
fig.append_trace(trace1, col = 1, row = 1)
fig.append_trace(trace2, col = 2, row = 1)
fig.append_trace(trace3, col = 1, row = 2)
fig.append_trace(trace4, col = 2, row = 2)
fig.update_xaxes(title='Groups', col=1, row=1)
fig.update_yaxes(title='Count', col=1, row=1)

fig.update_xaxes(title='Monetary', col=2, row=1)
fig.update_yaxes(title='Frequency', col=2, row=1)

fig.update_xaxes(title='Monetary', col=1, row=2)
fig.update_yaxes(title='Recency', col=1, row=2)

fig.update_xaxes(title='Frequency', col=2, row=2)
fig.update_yaxes(title='Recency', col=2, row=2)
fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))

fig.show()

# Сегментация с использованием машинного обучения

Было выполнено простое разбиение покупателей на 27 сегментов, для решения бизнес-задач такое количество сегментов не является оптимальным, так как слишком велико. Было принято решение уменьшить количество сегментов использовав метод кластеризации машинного обучения k-means.

In [20]:
df_result

,CustomerID,Recency,Frequency,Monetary,group_recency,group_frequancy,group_monetary,id_group,color
0,12346.0,325,2,0.00,3,1,1,311,1
1,12350.0,309,1,334.40,3,1,1,311,1
2,12353.0,203,1,89.00,3,1,1,311,1
3,12361.0,286,1,189.90,3,1,1,311,1
4,12373.0,310,1,364.60,3,1,1,311,1
...,...,...,...,...,...,...,...,...,...
4367,17440.0,147,5,177.10,3,3,1,331,251
4368,17912.0,309,6,295.61,3,3,1,331,251
4369,18037.0,153,6,70.02,3,3,1,331,251
4370,16321.0,71,7,230.70,2,3,1,231,261


Перед кластеризацией необходимо совершить стандартизацию данных, для уменьшения диапазона значений признаков, это в свою очередь поможет нам более чётко определить наши сегменты.

In [21]:
scaler = StandardScaler()
data_sc = scaler.fit_transform(df_result[['Recency', 'Frequency', 'Monetary']])

In [22]:
#L
data_sc

array([[ 2.32202285, -0.32936215, -0.23100099],
       [ 2.16322008, -0.43645506, -0.19031183],
       [ 1.11115173, -0.43645506, -0.22017164],
       ...,
       [ 0.61489308,  0.0990095 , -0.22248109],
       [-0.19897112,  0.20610242, -0.20292985],
       [-0.41732492, -0.00808341, -0.37279249]])

Обучим несколько моделей на сегментацию от 2 до 9 сегментов, сохраняя результат количества сегментов, суммы квадратов различий (SSE), и сами сегменты для каждого пользователя, в дальнейшем эти данные помогут принять финальное решение о количестве сегментов.

In [23]:
amount_clasters = []
inertia = []
clasters = []
for i in range(2,10):
  id_groups = df_result.groupby('id_group')['CustomerID'].count().reset_index().sort_values(by='CustomerID', ascending=False).head(i)['id_group'].values
  need_groups_values = df_result[df_result['id_group'].isin(id_groups)][['id_group', 'Recency','Frequency','Monetary']]
  centroids = np.array(need_groups_values.groupby('id_group').mean())
  kk = KMeans(n_clusters = len(centroids), init = np.array(centroids), n_init=1)
  data = kk.fit_predict(data_sc)
  inertia.append(kk.inertia_)
  amount_clasters.append(i)
  clasters.append(data)

Хорошая модель имеет низкую инерцию и небольшое количество сегментов, однако не стоит забывать о том, что с увеличением количества сегментов, инерция уменьшается. Для нахождения компромисса было решено использовать метод локтя. Построив график видим локоть(K) равный 3, 4, 6 и 7. Логически понимаем, что разбиение на 3 и 4 сегмента, не является оптимальным решением. Стоит рассмотреть такие варианты как 6 или же промежуточный вариант - 5.

In [24]:
df_gr = pd.DataFrame({'inertia': inertia, 'amount_clasters':amount_clasters})
fig = px.line(df_gr, x="amount_clasters", y="inertia")
fig.show()

Присоединим к нашему датафрейму, сегменты от 2 до 9, и посмотрим внимательнее, каким образом распределяются пользователи. А именно при наличии 5 и 6 сегментов. Оценим визуально, с помощью графика box plot, так же посмотрим на количественное распределение и среднии значения по каждому сегменту и признаку. На основании чего осуществим финальное решение о нашей сегментации

In [25]:
for i in range(0,7):
  df_result[amount_clasters[i]] = clasters[i]

In [26]:
df_result.groupby(5).agg({'CustomerID':'count',
                          'Recency':'mean',
                          'Frequency':'mean',
                          'Monetary':'mean'
                          })

,CustomerID,Recency,Frequency,Monetary
5,,,,
0,176,9.318182,29.670455,12793.610057
1,11,4.090909,109.909091,124312.306364
2,633,294.361769,1.507109,397.188167
3,2727,31.446645,4.696003,1424.619968
4,825,150.650909,2.423030,660.116741


In [27]:
df_result.groupby(6).agg({'CustomerID':'count',
                          'Recency':'mean',
                          'Frequency':'mean',
                          'Monetary':'mean'
                          })

,CustomerID,Recency,Frequency,Monetary
6,,,,
0,51,4.411765,44.921569,26861.954118
1,10,4.300000,116.300000,127930.999000
2,614,296.488599,1.501629,362.544642
3,476,15.846639,15.527311,4859.325021
4,2451,35.256222,3.488780,1058.427137
5,770,158.168831,2.431169,676.558612


In [28]:
n = 6
test = df_result.groupby(n).agg({'CustomerID':'count',
                          'Recency':'mean',
                          'Frequency':'mean',
                          'Monetary':'mean'
                          })
test['TotalCustomerID']   = [df_result.CustomerID.count() for _ in test.index]
test['TotalRecencyAvg']   = [df_result.Recency.mean()     for _ in test.index]
test['TotalFrequencyAvg'] = [df_result.Frequency.mean()   for _ in test.index]
test['TotalMonetaryAvg']  = [df_result.Monetary.mean()    for _ in test.index]

test['CustomerGain'] = test.CustomerID / test.TotalCustomerID
test['RecencyDeltaPerc']   = (test.Recency - test.TotalRecencyAvg) / test.Recency
test['FrequencyDeltaPerc'] = (test.Frequency - test.TotalFrequencyAvg) / test.Frequency
test['MonetaryDeltaPerc']  = (test.Monetary - test.TotalMonetaryAvg) / test.Monetary


test[['CustomerID', 'CustomerGain', 'Recency', 'RecencyDeltaPerc', 'Frequency', 'FrequencyDeltaPerc', 'Monetary', 'MonetaryDeltaPerc']].round(3)

,CustomerID,CustomerGain,Recency,RecencyDeltaPerc,Frequency,FrequencyDeltaPerc,Monetary,MonetaryDeltaPerc
6,,,,,,,,
0,51,0.012,4.412,-19.637,44.922,0.887,26861.954,0.929
1,10,0.002,4.300,-20.174,116.300,0.956,127930.999,0.985
2,614,0.140,296.489,0.693,1.502,-2.380,362.545,-4.236
3,476,0.109,15.847,-4.746,15.527,0.673,4859.325,0.609
4,2451,0.561,35.256,-1.582,3.489,-0.455,1058.427,-0.794
5,770,0.176,158.169,0.424,2.431,-1.088,676.559,-1.806


In [29]:
#цвета для 6 сегментов
segments = df_result[6].unique()
color_s = df_result['color'].unique()[0:6]
color_df = pd.DataFrame({6: segments, 'color_s': color_s})
df_result = df_result.merge(color_df, on=6)

In [30]:
bar_c = df_result.groupby(6)['CustomerID'].count().reset_index()

trace1 = go.Box(
y=df_result['Monetary'],
x=df_result[6],
name='Monetary'
)
trace2 = go.Box(
y=df_result['Frequency'],
x=df_result[6],
name='Frequency'
)
trace3 = go.Box(
y=df_result['Recency'],
x=df_result[6],
name='Recency',
#line=dict(color=df_result['color_s'].unique())
)
trace4 = go.Bar(
x=bar_c[6] ,
y=bar_c['CustomerID'],
name= 'Count',
marker=dict(color=df_result['color_s'].unique())
)

fig = make_subplots(rows=2, cols=2,  subplot_titles=('Monetary', 'Frequency', 'Recency', 'Count of buyers in the segment'))
fig.append_trace(trace1, row = 1, col = 1)
fig.append_trace(trace2, row = 1, col = 2)
fig.append_trace(trace3, row = 2, col = 1)
fig.append_trace(trace4, row = 2, col = 2)

fig.update_layout(height=700, width=1700)

fig.update_xaxes(title='Segments', col=1, row=1)
fig.update_yaxes(title='Purchases amount', col=1, row=1)

fig.update_xaxes(title='Segments', col=2, row=1)
fig.update_yaxes(title='Number of purchases', col=2, row=1)

fig.update_xaxes(title='Segments', col=1, row=2)
fig.update_yaxes(title='Amount of days', col=1, row=2)

fig.update_xaxes(title='Segments', col=2, row=2)
fig.update_yaxes(title='Number of users', col=2, row=2)

fig.show()

In [46]:
df_result

,CustomerID,Recency,Frequency,Monetary,group_recency,group_frequancy,group_monetary,id_group,color,2,3,4,5,6,7,8,color_s
0,12346.0,325,2,0.00,3,1,1,311,1,1,0,0,2,2,2,2,1
1,12350.0,309,1,334.40,3,1,1,311,1,1,0,0,2,2,2,2,1
2,12361.0,286,1,189.90,3,1,1,311,1,1,0,0,2,2,2,2,1
3,12373.0,310,1,364.60,3,1,1,311,1,1,0,0,2,2,2,2,1
4,12401.0,302,1,84.30,3,1,1,311,1,1,0,0,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4367,17811.0,3,51,7677.71,1,3,3,133,11,0,2,2,0,0,5,6,51
4368,17857.0,2,25,26763.34,1,3,3,133,11,0,2,2,0,0,5,6,51
4369,17949.0,0,52,52750.84,1,3,3,133,11,0,1,2,0,0,6,7,51
4370,14680.0,24,23,26932.34,2,3,3,233,41,0,2,2,0,0,5,6,51


Посмотрим на наши средние и медианные показатели по всем пользователям, без разделения на сегменты.

In [31]:
df_result[['Frequency', 'Monetary', 'Recency']].mean()

Frequency       5.075480
Monetary     1898.459701
Recency        91.047118
dtype: float64

In [32]:
df_result[['Frequency', 'Monetary', 'Recency']].median()

Frequency      3.000
Monetary     648.075
Recency       49.000
dtype: float64

In [33]:
df_result

,CustomerID,Recency,Frequency,Monetary,group_recency,group_frequancy,group_monetary,id_group,color,2,3,4,5,6,7,8,color_s
0,12346.0,325,2,0.00,3,1,1,311,1,1,0,0,2,2,2,2,1
1,12350.0,309,1,334.40,3,1,1,311,1,1,0,0,2,2,2,2,1
2,12361.0,286,1,189.90,3,1,1,311,1,1,0,0,2,2,2,2,1
3,12373.0,310,1,364.60,3,1,1,311,1,1,0,0,2,2,2,2,1
4,12401.0,302,1,84.30,3,1,1,311,1,1,0,0,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4367,17811.0,3,51,7677.71,1,3,3,133,11,0,2,2,0,0,5,6,51
4368,17857.0,2,25,26763.34,1,3,3,133,11,0,2,2,0,0,5,6,51
4369,17949.0,0,52,52750.84,1,3,3,133,11,0,1,2,0,0,6,7,51
4370,14680.0,24,23,26932.34,2,3,3,233,41,0,2,2,0,0,5,6,51


На графике отображено распределение 6 сегментов, рассматриваются такие признаки как частота и общая сумма покупок.

In [34]:
d = df_result[[6, 'Monetary', 'Frequency','Recency', 'color_s', 'CustomerID']]

trace1 = go.Scatter(x = d['Frequency'],
                    y = d['Recency'],
                    mode='markers',
                    marker=dict(color=d['color_s']), name = 'Frequency & Recency', hovertext=df_result[6])

trace2 = go.Scatter(x = d['Monetary'],
                    y = d['Frequency'],
                    mode = 'markers',
                    marker=dict(color=d['color_s']), name = 'Monetary & Frequency', hovertext=df_result[6])

trace3 = go.Scatter(x = d['Monetary'],
                    y = d['Recency'],
                    mode = 'markers',
                    marker=dict(color=d['color_s']), name = 'Monetary & Recency', hovertext=df_result[6])
trace4 = go.Bar(x=d[6].unique(), y = d.groupby(6)['CustomerID'].nunique(),  name='Counts of buyers in groups', marker=dict(color=d['color_s'].unique()))


fig = make_subplots(rows=2, cols=2, subplot_titles=('Frequency & Recency', 'Monetary & Frequency', 'Monetary & Recency', 'Number of buyers in segments'))
fig.append_trace(trace1, row = 1, col = 1)
fig.append_trace(trace2, row = 1, col = 2)
fig.append_trace(trace3, row = 2, col = 1)
fig.append_trace(trace4, row = 2, col = 2)

fig.update_xaxes(title='Frequency', col=1, row=1)
fig.update_yaxes(title='Recency', col=1, row=1)

fig.update_xaxes(title='Monetary', col=2, row=1)
fig.update_yaxes(title='Frequency', col=2, row=1)

fig.update_xaxes(title='Monetary', col=1, row=2)
fig.update_yaxes(title='Recency', col=1, row=2)

fig.update_xaxes(title='Segments', col=2, row=2)
fig.update_yaxes(title='Count', col=2, row=2)

fig.show()

# Финальные сегменты пользователей.
В результате было выбрано 6 сегментов пользователей, так как именно такое количество наилучшим образом отображает действительность.

**-- 0 сегмент Ядро**. вбирает в себя достаточно большое количество пользователей, который совершает заказы на суммы около общего среднего, и гораздо выше медианных значений по всем данным. Можно охарактеризовать таких пользователей как постоянные со средним чеком. Возможно заинтересовать допродажами, для увеличения их среднего чека.

**-- 1 сегмент Киты.** Совершают покупки на большие суммы, помимо прочего делают это очень часто. Нет необходимости прелагать им скидки, возможно заинтересовать их новинками.

**--2 Потерянные.** Последняя покупка была совершена в среднем год назад, низкий средний чек, не большая частота покупок, есть вероятность одноразовых клиентов. Возможно отправить рассылку напоминание о себе.

**--3 Спящие** сегмент достаточно крупный, пользователи которые не так часто совершают покупки, имеют в 2 раза ниже средний чек постоянных пользователей. Вероятно им стоит предложить программу лояльности, персональные скидки для того чтобы повысить их частоту покупок и средний чек и перевести в сегмент Ядра.

**--4 сегмент Уходящие.**Пользователи совершие последнюю покупку достаточно давно, небольшой средний чек, но еще возможно не потерянные, необходимо также напомнить о себе рассылкой, или обзвоном провести опрос по какой причине перестали покупать, постараться заинтересовать их.

**--5 сегмент новички с высоким чеком. Потенциальные VIP.**, совершают покупки достаточно часто, имеют высокий средний чек показатели в отрыве как от китов так и от ядра, рекомендуется изучить препочтения сегмента. Есть вероятность, что это новички, и при правильном взаимодействии станут постоянными пользователями.